In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

#PS: Pressure, bar, 100 Hz --> 100 Messungen pro Sekunde
#EPS: Motor power, W, 100 Hz 
#FS: Volume flow, l/min, 10 Hz --> 10 Messungen pro Sekunde
#TS: Temperature, Celsius, 1 Hz --> 1 Messung pro Sekunde
#VS: Vibration, mm/s, 1Hz
#CE: Cooling efficiency (virtual), %, 1 Hz
#CP: Cooling power (virtual), kW, 1 Hz
#SE: Efficency factor, %, 1 Hz


class DataProcessor:
    def __init__(self, input_path, file_names):
        self.input_path = input_path
        self.file_names = file_names
        
    def read_files(self):
        self.data = {}
        print("Reading files...")
        for file in self.file_names:
            with open(self.input_path + file + '.txt', 'r') as f:
                self.data[file] = pd.read_csv(f, header=None, sep='\t')
        return self.data
    
    def print_shape(self):
        print("Files read:")
        for file in self.data:
            print(f"{file}: {self.data[file].shape}")
            
    def create_target_df(self):
        target_columns = ['Cooler_Condition', 'Valve_Condition', 
                        'Internal_Pump_Leakage', 'Hydraulic_Accumulator', 
                        'Stable_Flag']
        self.data['target'].columns = target_columns
        self.valve_condition = self.data['target']['Valve_Condition']
        #del self.data['target']
        return self.valve_condition

def process_data():
    input_path = "input_data/"
    file_names = [
        "ce", "cp", "eps1", "se", "vs1", 
        "fs1", "fs2", 
        "ps1", "ps2", "ps3", "ps4", "ps5", "ps6",
        "ts1", "ts2", "ts3", "ts4", "target"
    ]
    
    processor = DataProcessor(input_path, file_names)
    data = processor.read_files()
    processor.print_shape()
    df_target = processor.create_target_df()
    df_target = processor.valve_condition
    return data, df_target

data, df_target = process_data()

Reading files...
Files read:
ce: (2205, 60)
cp: (2205, 60)
eps1: (2205, 6000)
se: (2205, 60)
vs1: (2205, 60)
fs1: (2205, 600)
fs2: (2205, 600)
ps1: (2205, 6000)
ps2: (2205, 6000)
ps3: (2205, 6000)
ps4: (2205, 6000)
ps5: (2205, 6000)
ps6: (2205, 6000)
ts1: (2205, 60)
ts2: (2205, 60)
ts3: (2205, 60)
ts4: (2205, 60)
target: (2205, 5)


In [14]:
import pandas as pd
from tsfel.feature_extraction import features
import tsfel

# Trennen von Input (X) und Target (y)
X = data['ce']
y = df_target

# tsfel Konfiguration für die Feature-Extraktion
cfg = tsfel.get_features_by_domain()  # Alle Domains
extracted_features = []

# Iteration über jede Zeile und Feature-Extraktion
for _, row in X.iterrows():
    features_row = tsfel.time_series_features_extractor(cfg, row.values)
    extracted_features.append(features_row)

# Features in einen DataFrame konvertieren
features_df = pd.DataFrame(extracted_features)

# Hinzufügen der Targets
features_df['Target'] = y.values

# Jetzt kannst du features_df für Modelltraining verwenden.


C:\Users\Daniel\AppData\Local\Temp\ipykernel_13124\808650864.py:15: UserWarning: Using default sampling frequency set in configuration file.
  features_row = tsfel.time_series_features_extractor(cfg, row.values)


C:\Users\Daniel\AppData\Local\Temp\ipykernel_13124\808650864.py:15: UserWarning: Using default sampling frequency set in configuration file.
  features_row = tsfel.time_series_features_extractor(cfg, row.values)


C:\Users\Daniel\AppData\Local\Temp\ipykernel_13124\808650864.py:15: UserWarning: Using default sampling frequency set in configuration file.
  features_row = tsfel.time_series_features_extractor(cfg, row.values)


C:\Users\Daniel\AppData\Local\Temp\ipykernel_13124\808650864.py:15: UserWarning: Using default sampling frequency set in configuration file.
  features_row = tsfel.time_series_features_extractor(cfg, row.values)


C:\Users\Daniel\AppData\Local\Temp\ipykernel_13124\808650864.py:15: UserWarning: Using default sampling frequency set in configuration file.
  features_row = tsfel.time_series_features_extractor(cfg, row.values)


...

C:\Users\Daniel\AppData\Local\Temp\ipykernel_13124\808650864.py:15: UserWarning: Using default sampling frequency set in configuration file.
  features_row = tsfel.time_series_features_extractor(cfg, row.values)


C:\Users\Daniel\AppData\Local\Temp\ipykernel_13124\808650864.py:15: UserWarning: Using default sampling frequency set in configuration file.
  features_row = tsfel.time_series_features_extractor(cfg, row.values)


C:\Users\Daniel\AppData\Local\Temp\ipykernel_13124\808650864.py:15: UserWarning: Using default sampling frequency set in configuration file.
  features_row = tsfel.time_series_features_extractor(cfg, row.values)


C:\Users\Daniel\AppData\Local\Temp\ipykernel_13124\808650864.py:15: UserWarning: Using default sampling frequency set in configuration file.
  features_row = tsfel.time_series_features_extractor(cfg, row.values)


ValueError: Must pass 2-d input. shape=(2205, 1, 155)

In [16]:
import numpy as np
from tsfel import time_series_features_extractor

# Fenstergröße festlegen
window_size = 10

# Sliding-Window-Ansatz
segments = []
targets = []

for i in range(0, len(X) - window_size + 1):
    segment = X.iloc[i:i + window_size].values.flatten()  # Zeilen im Fenster zusammenfügen
    segments.append(segment)
    targets.append(y.iloc[i + window_size - 1])  # Zielwert des letzten Punktes im Fenster

# Extrahieren der Features für jedes Segment
feature_list = []
for segment in segments:
    features_row = features.time_series_features_extractor(cfg, segment)
    feature_list.append(features_row)

# Features in DataFrame umwandeln
features_df = pd.DataFrame(feature_list)
features_df['Target'] = targets

# Jetzt kannst du features_df für Modelltraining verwenden.


AttributeError: module 'tsfel.feature_extraction.features' has no attribute 'time_series_features_extractor'

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(
    features_df.drop(columns=['Target']),
    features_df['Target'],
    test_size=0.2,
    random_state=42
)

# Modell trainieren
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Vorhersagen
predictions = model.predict(X_test)

<h1> tsfresh </h1>

In [24]:

from tsfresh import extract_features

# Beispiel-Daten: DataFrame laden
# data = pd.read_csv('your_data.csv')  # Falls du die Daten aus einer Datei hast

# DataFrame ins Long-Format transformieren
data_long = data['ps3'].reset_index().melt(id_vars="index", var_name="time", value_name="value")
data_long.rename(columns={"index": "id"}, inplace=True)

# Ergebnis:
# Spalten: ['id', 'time', 'value']


In [6]:
df_features = data['ps3']
df_features["time"] = df_features.index
df_features["id"] = df_features.index

In [4]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(df_target)

In [7]:
from tsfresh import extract_features
from tsfresh.feature_selection import select_features

# Merkmalsextraktion mit tsfresh
features = extract_features(df_features, column_id="id", column_sort="time", column_value="value")
features_cleaned = features.dropna(axis=1) 
# Feature-Selektion basierend auf Zielwerten
selected_features = select_features(features_cleaned, y=y_encoded)  # labels = Zielvariablen (falls vorhanden)


ValueError: Column not found: value

In [20]:
df_features = pd.DataFrame(selected_features)

In [ ]:
df_features

In [32]:
df_features.replace([np.inf, -np.inf], np.nan, inplace=True)

In [33]:
df_features = df_features.dropna(how = "all", axis= "columns")

In [34]:
df_features = df_features.ffill(axis="index")

In [ ]:
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="white")

# Compute the correlation matrix
corr = df_features.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})


In [22]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold()
df_features = selector.fit_transform(df_features)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

  
states = [27, 6728, 49122]
accs = []
features = df_features
target = y_encoded

for RANDOM_STATE in states:
    X_train, X_test, y_train, y_test = train_test_split(
        features, target, test_size = 0.2, random_state = RANDOM_STATE, stratify = target
    )
    
    model = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=50
    )
    
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    accs.append(accuracy_score(y_test, preds))
    print(f"Random State: {RANDOM_STATE}")
    print(classification_report(y_test, preds, zero_division=0.0))

accs_mean = round(np.mean(accs), 4)
accs_std = round(np.std(accs), 4)

print(f"Mean Accuracy: {accs_mean}")
print(f"Std Accuracy: {accs_std}")